In [165]:
import pandas as pd

In [166]:
df = pd.read_csv("retouren.csv")

# 1. Data Cleaning

In [ ]:
df.describe().round(1)

,order_item_id,item_id,brand_id,item_price,user_id,return
count,75007.0,75007.0,75007.0,75007.0,75007.0,75007.0
mean,37504.0,784.8,28.8,60.7,25740.2,0.5
std,21652.8,680.0,26.8,44.7,12947.2,0.5
min,1.0,1.0,1.0,0.0,9.0,0.0
25%,18752.5,176.0,5.0,29.9,14674.0,0.0
50%,37504.0,479.0,24.0,49.9,30486.0,0.0
75%,56255.5,1517.0,43.0,74.9,36479.0,1.0
max,75007.0,2224.0,137.0,999.0,43801.0,1.0


In [ ]:
df.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return
0,1,2016-06-22,2016-06-27,643,38,navy,30,49.90,30822,Mrs,1969-04-17,Saxony,2016-06-23,0
1,2,2016-06-22,NaN,337,152,grey,30,19.95,30822,Mrs,1969-04-17,Saxony,2016-06-23,0
2,3,2016-06-22,2016-06-27,270,xxl,grey,49,79.90,30823,Mrs,1970-04-22,Baden-Wuerttemberg,2015-03-15,1
3,4,2016-06-22,2016-06-27,142,xxl,grey,49,99.90,30823,Mrs,1970-04-22,Baden-Wuerttemberg,2015-03-15,0
4,5,2016-06-22,2016-06-27,561,xxl,grey,3,14.90,30823,Mrs,1970-04-22,Baden-Wuerttemberg,2015-03-15,1


### Dtypes

In [175]:
df.dtypes

order_item_id      int64
order_date        object
delivery_date     object
item_id            int64
item_size         object
item_color        object
brand_id           int64
item_price       float64
user_id            int64
user_title        object
user_dob          object
user_state        object
user_reg_date     object
return             int64
dtype: object

- **order_date, delivery_date, user_dob als datetime**

In [179]:
df.delivery_date = pd.to_datetime(df.delivery_date)
df.order_date = pd.to_datetime(df.order_date)
df.user_dob = pd.to_datetime(df.user_dob)

### Item_id's mit item_price von 0 

In [ ]:
df[df.item_price == 0].item_id.unique()

array([ 157,   62,  312,  553,  807, 1014,  239, 1436, 1725, 1904, 1951,
       1472, 2025, 2103, 2121, 2199, 2213])

In [ ]:
df[df.item_id == 157][["item_color", "item_price"]]

,item_color,item_price
202,purple,0.0
216,purple,0.0
218,purple,0.0
329,purple,9.9
351,purple,0.0
...,...,...
65632,jade,9.9
65943,jade,9.9
66731,jade,9.9
68026,jade,9.9


In [ ]:
df[df.item_id == 157][["item_color", "item_price"]].groupby("item_color").agg(
 mean_price = ("item_price", "mean"),
 max_price= ("item_price", "max"),
 min_price = ("item_price", "min")
)

,mean_price,max_price,min_price
item_color,,,
jade,10.710811,19.9,9.9
purple,1.218436,19.9,0.0


In [ ]:
df[(df.item_id == 157) & (df.item_color == "purple")][["item_color", "item_price"]].item_price.unique()

array([ 0. ,  9.9, 19.9])

In [ ]:
df[df.item_id == 157][["item_color", "item_price"]].groupby(["item_color", "item_price"]).agg(
 item_price_count = ("item_price", "count")
)

item_price_count
item_color item_price                  
jade       9.9                       34
           19.9                       3
purple     0.0                      160
           9.9                       16
           19.9                       3

- **für die item_id 157 gibt es verschiedene Preise** 
- **Promotion Angebot zB. 2 für 3**

-> **nicht in df ändern**


### Delivery_date NaN

In [ ]:
df[df.delivery_date.isna()][["order_date", "delivery_date", "user_id"]]

,order_date,delivery_date,user_id
1,2016-06-22,NaN,30822
55,2016-06-23,NaN,30828
63,2016-06-23,NaN,22948
64,2016-06-23,NaN,22948
65,2016-06-23,NaN,22948
...,...,...,...
74831,2016-08-20,NaN,43763
74839,2016-08-20,NaN,38972
74924,2016-08-20,NaN,19160
74925,2016-08-20,NaN,19160


In [ ]:
df[df.delivery_date.isna()][["order_date", "delivery_date", "user_id"]].nunique()

order_date         60
delivery_date       0
user_id          4048
dtype: int64

In [ ]:
df_1 = df.copy()

In [ ]:
df[df.delivery_date.isna()][["order_date", "delivery_date", "user_id"]].user_id.unique()

array([30822, 30828, 22948, ..., 16813, 43763, 19160])

In [ ]:
df_1.delivery_date = df_1.delivery_date.fillna("NaN")

In [ ]:
df_d = df_1.groupby(["user_id", "delivery_date"]).agg(
 count = ("delivery_date", "count")
).reset_index()

In [ ]:
df_nan = df_d[df_d.delivery_date == "NaN"].user_id

In [ ]:
df_d[df_d.user_id.isin(df_nan)]

,user_id,delivery_date,count
8,55,2016-07-03,2
9,55,2016-08-08,8
10,55,2016-08-23,1
11,55,2016-10-03,1
12,55,NaN,1
...,...,...,...
30368,43728,2016-08-24,5
30369,43728,2016-08-30,1
30370,43728,NaN,3
30397,43763,2016-08-23,1


- **für NaN werte bei delivery_date not delivered eintragen (behalten)** 
- **falls das Modell soll vor der Bestellung eingesetzt werden spielt das delivery_date noch keine Rolle (weglassen)**
- 
  
  